# HW-aware Synthesis of MCx

This example shows that implementation of multiple control-x (MCX) logic, using the Classiq synthesis engine, yields different circuit results for different quantum hardware.

The fictitious hardware created here demonstrates how to insert your own custom-designed machine. For comparison, create two types of hardware with `cx, u` basis gates. The difference between them manifests in the connectivity map: one has linear connectivity while the other has all-to-all connectivity.

In [1]:
from classiq import Model, show, synthesize
from classiq.builtin_functions import Mcx
from classiq.model import Constraints, CustomHardwareSettings, Preferences

# define the Mcx parameters

mcx_params = Mcx(num_ctrl_qubits=15)
max_width = 18

# define the hardware parameters

linear_connectivity = [[qubit, qubit + 1] for qubit in range(max_width - 1)]

preferences_linear = Preferences(
    custom_hardware_settings=CustomHardwareSettings(
        basis_gates=["cx", "u"],
        connectivity_map=linear_connectivity,
    ),
    random_seed=-1,
)
preferences_all_to_all = Preferences(
    custom_hardware_settings=CustomHardwareSettings(basis_gates=["cx", "u"]),
    random_seed=-1,
)

# create the quantum circuits
model_linear = Model(
    preferences=preferences_linear,
    constraints=Constraints(optimization_parameter="depth", max_width=max_width),
)
model_linear.Mcx(mcx_params)

model_all_to_all = Model(
    preferences=preferences_all_to_all,
    constraints=Constraints(optimization_parameter="depth", max_width=max_width),
)
model_all_to_all.Mcx(mcx_params)

# synthesize and present the circuit results

qmod_linear = model_linear.get_model()
with open("hardware_aware_mcx_linear.qmod", "w") as f:
    f.write(qmod_linear)
linear_result = synthesize(qmod_linear)

qmod_all_to_all = model_all_to_all.get_model()
with open("hardware_aware_mcx_all_to_all.qmod", "w") as f:
    f.write(qmod_all_to_all)
all_to_all_result = synthesize(qmod_all_to_all)

show(linear_result)
show(all_to_all_result)

Opening: https://platform.classiq.io/circuit/1d0bbb60-ad11-46f0-b555-44d87610ba3b?version=0.34.0.dev0%2B124706fb0e
Opening: https://platform.classiq.io/circuit/812a9496-625b-4cae-a1f2-41f27adf9a75?version=0.34.0.dev0%2B124706fb0e


Comparison of the two circuits shows that applying MCx using different connectivity maps yields different implementation.

Using "all-to-all" connectivity, the synthesis engine chooses as the best implementation a recourse based on "Maslov2015" [[1](#Maslov)] that was written on the Classiq platform. Using that, the manufactured circuit has 18 qubits; i.e., it uses two auxiliary qubits. The total depth of the circuit is 828.

When using linear connectivity, the best implementation chosen by the synthesis engine is, in fact, different: an algorithm developed by Classiq, which is better suited for this map. Here, the manufactured circuit uses 17 qubits with only one auxiliary and has a depth of 1679 gates.

## References

<a id='Maslov'>[1]</a>: [Maslov, D., 2016. Advantages of using relative-phase Toffoli gates with an application to multiple control Toffoli optimization. Physical Review A, 93(2), p.022311.](https://arxiv.org/pdf/1508.03273.pdf)